In [165]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from sklearn.cluster import KMeans

# Question 1

## **Part 1.1: Work**



#### Read Data

In [155]:
PATH_TO_Q1_DATA = 'data/.csv'
df = pd.read_csv(PATH_TO_Q1_DATA) 

In [156]:
df.head()

,S IgG1,RBD IgG1,N IgG1,S IgG2,RBD IgG2,N IgG2,S IgG3,RBD IgG3,N IgG3,S IgG4,...,NT50,Sex,Race,Age,Remdesivir,Antibiotics,Chloroquines,Tocilizumab,Ards,Outcome
0,15518.25,11686.50,17408.00,2595.00,2977.5,4418.75,7988.75,10112.75,13740.75,2890.50,...,264.0,Male,Black,60-69,No,Yes,Yes,No,Yes,Died
1,2226.50,2343.25,3379.75,2444.75,3850.5,4687.00,2404.00,3399.00,4383.00,2214.75,...,126.0,Male,White,60-69,No,Yes,Yes,No,Yes,Died
2,2279.75,2450.00,3741.00,2117.00,2321.0,3298.50,2110.75,2256.25,3009.75,2256.75,...,16.0,Female,Asian,80 and older,Yes,No,No,No,No,Died
3,115548.25,93832.00,94973.50,2938.75,2725.0,4380.25,17165.50,14912.00,33445.50,2512.25,...,53.0,Male,Asian,50-59,Yes,Yes,No,No,Yes,Lived
4,2645.00,2479.00,4630.00,2256.75,2408.5,3449.50,2086.00,2452.50,3574.75,2127.00,...,151.0,Female,White,80 and older,Yes,No,No,No,No,Lived


#### Standardize Data in Columns 1-52

In [161]:
'''
 TODO: Standardize columns 1-52 by subtracting off mean of each column and scaling to unit variance
 
'''   
cols = df.iloc[:,:52].columns # get list of columns 1-52
for col in cols:
    if '_zscore' not in col:
        col_zscore = col + '_zscore' # add column for zscore
    df[col_zscore] = (df[col] - df[col].mean())/ df[col].std() # z =(x-μ)/σ  #??std(ddof=??) degrees of freedom

In [162]:
df.head(1)

,S IgG1,RBD IgG1,N IgG1,S IgG2,RBD IgG2,N IgG2,S IgG3,RBD IgG3,N IgG3,S IgG4,...,S ADCD_zscore,RBD ADCD_zscore,N ADCD_zscore,S NKD-CD107a_zscore,RBD NKD-CD107a_zscore,N NKD-CD107a_zscore,S NKD-MIP1b_zscore,RBD NKD-MIP1b_zscore,N NKD-MIP1b_zscore,NT50_zscore
0,15518.25,11686.5,17408.0,2595.0,2977.5,4418.75,7988.75,10112.75,13740.75,2890.5,...,-0.675782,-0.411877,0.450513,-0.02381,-0.180116,-0.576487,-0.586417,-1.100901,-0.542606,1.019302


#### Cluster Standardized Data in Columns 1-52

In [166]:
# 1 < k < 11
possible_cluster_nums = [2,3,4,5,6,7,8,9,10]

for k in possible_cluster_nums:
  '''
    TODO: 1. Fit data to k clusters using imported clustering algorithm
          2. Compute quaility of results for k clusters using imported
             cluster quality metric and store in a list
  '''

kmeans = KMeans(n_clusters=5).fit(df)

ValueError: could not convert string to float: 'Male'

#### Visualize Cluster Quality Metrics

In [ ]:
# Creates line chart to visualize values of cluster quality metric for each possible number of clusters
def plotMetricByK(metric_name, metric_results, PATH_TO_SAVE=None):
  '''
  metric_name: Name of cluster quality metric for title and axis label
  metric_results: List containing value of metric, in order, for each
                  possible number of clusters
  PATH_TO_SAVE: Path of file to save plot. If path is not provided, image is not saved
  '''

  plt.clf()

  plt.title(f'{metric_name} by Number of Clusters')
  plt.xlabel('Number of Clusters')
  plt.ylabel(metric_name)
  plt.plot(possible_cluster_nums, metric_results)
  
  if PATH_TO_SAVE:
    plt.savefig(PATH_TO_SAVE)

In [ ]:
'''
 TODO: Plot your choice of cluster quality metric by cluster number to help determine k.

 TIP: If using the above function, place each function call in a separate 
      cell to visualize multiple cluster quality metrics
'''

## **Part 1.1: Answer**

How many clusters are there in the data? **YOUR ANSWER HERE** 

Explanation: **Please make sure any relevant plots are either included in the above cells or embedded in this cell and replace this line with a brief explanation of how they justify your choice**



## **Part 1.2: Work**

#### Cluster Data

In [ ]:
'''
  TODO: Cluster data with the number of clusters you determined in part 1.1 and store resulting labels
'''

#### Univariate Analysis

In [ ]:
'''
  TODO: Find 4 variables that have statistically significant differences between values in the clusters (i.e p < 0.05 using pairwise t-tests)
'''

## **Part 1.2: Answer**

VARIABLE 1, VARIABLE 2, VARIABLE 3, VARIABLE 4

**Replace the above line with 4 variables that you found from the univariate analysis and output the corresponding box plots in the cells below or as images in this cell**

In [ ]:
'''
  TODO: Create and display boxplots for each of the 4 variables like those in Fig 1. from the paper by Wu et al

  TIP: 1. If you add the cluster labels as an additional column to the dataframe, then you can follow the example at https://www.pythonprogramming.in/boxplot-group-by-column-data.html
       2. If you want to use subplots to put all 4 boxplots in the same figure, you can plot the dataframes on a specific subplot using the ax keyword. For example,
          
          fig, axs = plt.subplots(2, 2)

          df.boxplot(column=['Variable'], by=['Cluster'], ax=axs[0,0])
          df.boxplot(column=['Variable'], by=['Cluster'], ax=axs[0,1])
''' 

## **Part 1.3: Work**


In [ ]:
# Formats Pandas series to string of form 'index_1: value_1, ..., index_n:, value_n' w/ indexes alphabetically sorted
def formatValueCounts(value_counts):
  '''
    value_counts: Pandas series
  '''
  count_string = value_counts.to_string()
  formatted_counts = [': '.join(count.split()) for count in count_string.split('\n')]
  formatted_counts.sort()
  return ', '.join(formatted_counts)

# Creates table summarizing data by cluster and categorical feature
def plotSummaryTable(cellText, PATH_TO_SAVE=None):
  '''
  cellText: num_clusters x 9 2D List where cellText[i][j] contains a string summarizing
            the statistics for cluster i and column (53 + j) in the data
  PATH_TO_SAVE: Path of file to save plot. If path is not provided, image is not saved
  '''

  k = len(cellText)
  colLabels = [f'Cluster {i + 1}' for i in range(k)]

  rowLabels = list(df.columns)[52:61]

  cellText = np.array(cellText).T

  plt.figure(figsize=(10,10))
  table = plt.table(cellText, 
              colLabels=colLabels,
              colColours=['#D3D3D3'] * len(colLabels),
              rowLabels=rowLabels,
              rowColours=['#D3D3D3'] * len(rowLabels),
              cellLoc='center',
              loc='upper center')
  table.scale(2,5)
  table.auto_set_font_size(False)
  table.set_fontsize(12)

  plt.axis('off')
  plt.grid(False)

  if PATH_TO_SAVE:
    plt.savefig(PATH_TO_SAVE)

In [ ]:
'''
  TODO: Create a table where the rows correspond to the variables in columns 53-61, and the columns correspond to the k clusters you identified.
        For each cell in the table, put summary statistics for that (variable, cluster) pair

  TIP: 1. If you create a 2D list, cellText, where cellText[i][j] contains a string summarizing the statistics for cluster i and column (53 + j) in
          the data, you can pass this into plotSummaryTable (provided above) to automatically create the table with matplotlib
       2. If you use value_counts() from Pandas, you can pass the resulting series to formatValueCounts (provided above) to convert it to a formatted string
'''

## **Part 1.3: Answer**

**Plot the table in one of the above cells or include it as an image in this cell**

Are any of the clusters significantly enriched for some particular value? **YOUR ANSWER HERE**



## **Part 1.4: Work**

In [ ]:
'''
TODO: Cluster the numeric variables (Columns 1-52)
'''

## **Part 1.4: Answer**

How many clusters are there in the numeric variables? **YOUR ANSWER HERE** 

Explanation: **Please make sure any relevant plots are either included in the above cells or embedded in this cell and replace this line with a brief explanation of how they justify your choice**

## **Part 1.5: Work**

In [ ]:
'''
TODO: 1. Choose a representative variable from each cluster you determined in Part 1.4
         and create a low-dimensional version of the data using those variables
      2. Re-cluster the data using the reduced representation using the same 
         choices you made for part 1.1.
'''

## **Part 1.5: Answer**

Representative Variables: **Replace this with the representative variables you chose using your work from Part 1.4** 

How many clusters are there in the numeric variables? **YOUR ANSWER HERE** 

Explanation: **Please make sure any relevant plots are either included in the above cells or linked in this cell and replace this line with a brief explanation of how they justify your choice**

## **Part 1.6: Work**

In [ ]:
'''
  TODO: Create a table where the rows correspond to the variables in columns 53-61, and the columns correspond to the k clusters you identified.
        in part 1.5. For each cell in the table, put summary statistics for that (variable, cluster) pair
'''

## **Part 1.6: Answer**

**Plot the table in one of the above cells or include it as an image in this cell**

Are any of the clusters significantly enriched for some particular value? **YOUR ANSWER HERE**